In [1]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
dir <- "environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees"
suppressPackageStartupMessages(source("tree_helper_function.R"))
load("environment/brain_sim_nodtu/mode=gc_bias/mapDf.RData")
load(file.path(dir, "treeCons.RData"))
load(file.path(dir, "yAggConsNS.RData"))
load(file.path(dir, "treeCor.RData"))
load(file.path(dir, "yAggCorNS.RData"))
load(file.path(dir, "treeCons0.RData"))
load(file.path(dir, "yAggConsNS0.RData"))
load(file.path(dir, "treeMeanInf0.RData"))
load(file.path(dir, "yAggMeanInfNS0.RData"))
load(file.path(dir, "treeThr.RData"))
load(file.path(dir, "yAggThrNS.RData"))
suppressPackageStartupMessages(source("brain_simulation_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/tree_analysis/tree_filter.R"))

In [ ]:
dfDepth <- data.frame(matrix(nrow=0, ncol=4))
colnames(dfDepth) <- c("Type", "Value", "nCuts", "gamma")

l <- length(treeCons$tip)
trees <- list(treeCons, treeCons0, treeMeanInf0, treeCor, treeThr)
names(trees) <- c("Consensus", "Consensus0", "MeanInf0", "Cor", "Thr")
yAgg <- list(yAggConsNS, yAggConsNS0, yAggMeanInfNS0, yAggCorNS, yAggThrNS)
names(yAgg) <- c("Consensus", "Consensus0", "MeanInf0", "Cor", "Thr")

dCutsDLFC <- vector(mode = "list", length=length(yAgg))
names(dCutsDLFC) <- paste("gamma=", c(0.1,0.5,1,5,10), sep="")
for(j in seq_along(dCutsDLFC)) {
    gamma <- c(0.1,0.5,1,5,10)[j]
    dCutsDLFC[[j]] <- vector(mode="list", length(trees))
    names(dCutsDLFC[[j]]) <- names(trees)
    for(i in seq(trees)) {
        globArr <- rep(-100, l+trees[[i]]$Nnode)
        descL <- sapply(Descendants(trees[[i]], seq(l+trees[[i]]$Nnode)), length)
        infReps <- assays(yAgg[[i]])[grep("infRep", assayNames(yAgg[[i]]))]
        infReps <- abind::abind(as.list(infReps), along = 3)
        lfc <- getLog2FC(infReps, colData(yAgg[[i]])[["condition"]])
        obj <- abs(lfc/mcols(yAgg[[i]])[["meanInfRV"]]) - gamma*node.depth(trees[[i]],2)
        val <- findMaxSum(trees[[i]], obj, l+1, descL)
        gc()
        dCutsDLFC[[j]][[i]] <- findCuts(trees[[i]], globArr, obj, l+1, descL)
        gc()
        dfDepth <- rbind(dfDepth, data.frame(Type = names(trees)[i], Value = val,
               nCuts = length(dCutsDLFC[[j]][[i]]), gamma=gamma))
    }
}
print(dfDepth)
save(dfDepth, file=file.path(dir, "dfLFC.RData"))
save(dCutsDLFC, file=file.path(dir, "dCutsDLFC.RData"))